# Imports

In [21]:
from urllib.parse import urlparse, parse_qs, urlencode
from requests import get, post
from getpass import getpass
from re import findall

# Get token

## Urls def

In [3]:
beatport_base_url = 'https://api.beatport.com/v4'
beatport_redirect_uri = beatport_base_url + '/auth/o/post-message/'

## Get client id

In [11]:
client_id = ''
html = get('https://api.beatport.com/v4/docs/').content.decode('utf-8')
scripts_matches = findall(r"src=.(.*js)", html)
for script_url in scripts_matches:
    url = 'https://api.beatport.com{}'.format(script_url)
    js = get(url.format(script_url)).content.decode('utf-8')
    client_id_matches = findall(r"API_CLIENT_ID: \'(.*)\'", js)
    if client_id_matches:
        client_id = client_id_matches[0]
client_id

0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd


## Login

In [20]:
response = post(url=beatport_base_url + '/auth/login/', 
                json={'username': input("Enter your username: "),
                      'password': getpass("Enter your password: ")})
response.json()

{'username': 'MrWr',
 'first_name': 'CortoZ',
 'last_name': 'Deschamp',
 'email': 'corto.deschamps@cpe.fr',
 'email_confirmed': False,
 'register_ip_address_ntoa': '77.199.189.52',
 '_next': 'https://www.beatport.com'}

## Authorization code

In [22]:
urlencode({
    'response_type': 'code',
    'client_id': client_id,
    'redirect_uri': beatport_redirect_uri
})

'response_type=code&client_id=0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd&redirect_uri=https%3A%2F%2Fapi.beatport.com%2Fv4%2Fauth%2Fo%2Fpost-message%2F'

In [25]:
query = urlencode({
    'response_type': 'code',
    'client_id': client_id,
    'redirect_uri': beatport_redirect_uri
})
response = get(url=f'{beatport_base_url}/auth/o/authorize/?{query}', allow_redirects=False)
response.headers

{'server': 'envoy', 'date': 'Mon, 08 Jan 2024 09:57:05 GMT', 'content-type': 'text/html; charset=utf-8', 'Content-Length': '0', 'location': '/v4/auth/login/?next=/v4/auth/o/authorize/%3Fresponse_type%3Dcode%26client_id%3D0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd%26redirect_uri%3Dhttps%253A%252F%252Fapi.beatport.com%252Fv4%252Fauth%252Fo%252Fpost-message%252F', 'x-frame-options': 'DENY', 'vary': 'origin, Authorization, Cookie', 'x-content-type-options': 'nosniff', 'referrer-policy': 'same-origin', 'x-envoy-upstream-service-time': '11', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}